In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
movies = pd.read_csv('dataset/tmdb_5000_movies.csv')
credits = pd.read_csv('dataset/tmdb_5000_credits.csv')

# Data Pre-processing

In [3]:
movies = movies.merge(credits, on = 'title')
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.dropna(inplace=True)

In [4]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


## Merging all important textual info into a single list.

### Converting stringified lists/dicts into Python lists.


Extracting each genre/keyword name into a simple list.

In [5]:
import ast

def convert(text):
    l = []
    for i in ast.literal_eval(text):
        l.append(i['name'])
    return l

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

Extracting top 3 cast members

In [6]:
def convert_cast(text):
    l = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            l.append(i['name'])
        counter += 1
    return l

movies['cast'] = movies['cast'].apply(convert_cast)

Finding and extracting only the director's name from the crew.

In [7]:
def fetch_director(text):
    l = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

movies['crew'] = movies['crew'].apply(fetch_director)

### Splitting overview into individual words

Tokenizing the movie overview text.

In [8]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

### Removing spaces inside multi-word feature entries

This is important for vectorization

In [9]:
def remove_space(word):
    l = []
    for i in word:
        l.append(i.replace(' ',''))
    return l

movies['keywords'] = movies['keywords'].apply(remove_space)
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)


In [10]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']


## Creating dataframe to store only essential columns for recommendation.

In [11]:
new_df = movies[['movie_id', 'title', 'tags']]

new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [12]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


## Apply stemming to reduces words to root forms.

Reducing vocabulary size improves vectorization.

In [13]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

def stems(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))
    return ' '.join(l)

new_df['tags'] = new_df['tags'].apply(stems)

### Converting text to numeric vectors

Creates a Bag-of-Words matrix with 5,000 most common words.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new_df['tags']).toarray()


### Compute similarity between movies

Measures how similar two movies are based on their tags.

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)


# Recommendation function

*   Gets the index of the selected movie.
*   Retrieves its similarity scores with all movies.
*   Sorts movies based on similarity score.
*   Skips the first one (because it's the same movie)
*   Prints top 5 most similar movies




In [16]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),
                       reverse=True,
                       key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)


### Eg

In [17]:
recommend('Spider-Man')

Spider-Man 3
Spider-Man 2
The Amazing Spider-Man 2
Arachnophobia
Kick-Ass


### Saving data to pickle files

Saving the processed data and similarity matrix to disk.so they can be loaded in another script.

In [18]:
import pickle
pickle.dump(new_df, open('movie_list.pkl', 'wb'))
pickle.dump(similarity, open('similarity_list.pkl', 'wb'))